In [1]:
from source import data_import
from source.constants import CHANNELS
import pandas as pd


In [2]:
# p_df = data_import.import_patients(patient_ids=[1, 2, 3, 4], seizure_flag=True)
# p_df.head()

In [3]:
# data_import.save_pyarrow(p_df, file_name="preprocess_test")

In [4]:
# import segmented data
s_df = data_import.load_segmented_data(patient_ids=[1,2])

chb01_01.edf was import but not resampled 256Hz.
chb01_02.edf was import but not resampled 256Hz.
chb01_03.edf was import but not resampled 256Hz.
chb01_03.edf seizure and buffer was labeled
seizure end reached. epoch number: 2, number of segments completed: 2
adding 2 ictal segments to epoch 2.
adding 13 pre-ictal segments to epoch 2.
chb01_04.edf was import but not resampled 256Hz.
chb01_04.edf seizure and buffer was labeled
seizure end reached. epoch number: 3, number of segments completed: 1
adding 1 ictal segments to epoch 3.
adding 14 pre-ictal segments to epoch 3.
chb01_05.edf was import but not resampled 256Hz.
chb01_06.edf was import but not resampled 256Hz.
chb01_07.edf was import but not resampled 256Hz.
chb01_08.edf was import but not resampled 256Hz.
chb01_09.edf was import but not resampled 256Hz.
chb01_10.edf was import but not resampled 256Hz.
chb01_11.edf was import but not resampled 256Hz.
chb01_12.edf was import but not resampled 256Hz.
chb01_13.edf was import but no

In [5]:
data_import.save_pyarrow(s_df, file_name="preprocess_seg")

/home/weasel/reps/ai-seizure-detectives/source/../data/preprocess_seg.arrow was successfully written.


In [6]:
# p_df = data_import.load_pyarrow(file_name="preprocess_test")
p_df = data_import.load_pyarrow(file_name="preprocess_seg")

/home/weasel/reps/ai-seizure-detectives/source/../data/preprocess_seg.arrow was loaded.


In [7]:
from source.filter_eeg_channels import filter_eeg_channels

exclude_ranges=[[58, 62], [118, 122]]
fit_df = filter_eeg_channels(p_df, CHANNELS, fs=256, exclude_ranges=exclude_ranges, Q=30)
fit_df.head()

channel,F4-C4,F3-C3,FT9-FT10,FZ-CZ,F7-T7,FP2-F4,T8-P8-1,T8-P8-0,FP1-F3,CZ-PZ,before_seizure,is_seizure
0 days 00:30:00,16.878726,20.334213,-3.322584,17.941953,18.739373,33.092935,12.625819,12.625819,44.788430,2.790970,False,False
0 days 00:30:00.003906250,22.891349,24.865283,-8.175141,23.784967,26.448730,40.639022,11.874291,11.874291,59.506243,4.538652,False,False
0 days 00:30:00.007812500,22.162881,25.360125,-8.902114,25.615925,27.023819,40.074853,5.285063,5.285063,60.136641,5.292160,False,False
0 days 00:30:00.011718750,29.573426,33.953147,-14.930178,33.038895,32.700204,52.956037,8.308854,8.308854,79.842157,8.558853,False,False
0 days 00:30:00.015625,21.292400,18.163275,-11.955401,21.366748,20.938086,35.328903,-1.132117,-1.132117,60.607725,6.096930,False,False


In [8]:
### go through non-seizure data (before_seizure)
# extract seizure-starts
    # for each seizure start
        # get previous 20 seconds
        # check if is_seizure in it AND if all are before_seizure
            # yes: drop and continue to next seizure start
        # take 20 seconds:
            # mean of amplitude over time per channel
            # mean of power spectrum for frequency ranges per channel

### go through seizure data (is_seizure)
# take seizre-starts
    # for each seizure start
        # get next 20 seconds
        # check if all are is_seizure
            # no: drop and continue
        # take 20 seconds:
            # mean of amplitude over time per channel
            # mean of power spectrum for frequency ranges per channel

# out put: data resampled to 20sec with features

In [9]:
fit_df.info()

<class 'pandas.core.frame.DataFrame'>
TimedeltaIndex: 5836800 entries, 0 days 00:30:00 to 0 days 00:34:59.996093750
Data columns (total 12 columns):
 #   Column          Dtype  
---  ------          -----  
 0   F4-C4           float64
 1   F3-C3           float64
 2   FT9-FT10        float64
 3   FZ-CZ           float64
 4   F7-T7           float64
 5   FP2-F4          float64
 6   T8-P8-1         float64
 7   T8-P8-0         float64
 8   FP1-F3          float64
 9   CZ-PZ           float64
 10  before_seizure  bool   
 11  is_seizure      bool   
dtypes: bool(2), float64(10)
memory usage: 501.0 MB


In [10]:
fit_df['seizure_start'] = fit_df['is_seizure'] & ~fit_df['is_seizure'].shift(fill_value=False)
fit_df.loc[fit_df['seizure_start'] == True, 'seizure_start']

0 days 00:49:55    True
0 days 00:24:26    True
0 days 00:28:51    True
0 days 00:16:54    True
0 days 00:28:39    True
0 days 00:05:26    True
0 days 00:31:01    True
0 days 00:49:31    True
Name: seizure_start, dtype: bool

In [11]:
fit_df['seizure_id'] = fit_df['seizure_start'].cumsum()
fit_df['seizure_id'].tail()

0 days 00:34:59.980468750    8
0 days 00:34:59.984375       8
0 days 00:34:59.988281250    8
0 days 00:34:59.992187500    8
0 days 00:34:59.996093750    8
Name: seizure_id, dtype: int64

In [12]:
fit_df.index

TimedeltaIndex([          '0 days 00:30:00', '0 days 00:30:00.003906250',
                '0 days 00:30:00.007812500', '0 days 00:30:00.011718750',
                   '0 days 00:30:00.015625', '0 days 00:30:00.019531250',
                '0 days 00:30:00.023437500', '0 days 00:30:00.027343750',
                   '0 days 00:30:00.031250', '0 days 00:30:00.035156250',
                ...
                '0 days 00:34:59.960937500', '0 days 00:34:59.964843750',
                   '0 days 00:34:59.968750', '0 days 00:34:59.972656250',
                '0 days 00:34:59.976562500', '0 days 00:34:59.980468750',
                   '0 days 00:34:59.984375', '0 days 00:34:59.988281250',
                '0 days 00:34:59.992187500', '0 days 00:34:59.996093750'],
               dtype='timedelta64[ns]', length=5836800, freq=None)

In [15]:
data_import.save_pyarrow(fit_df, file_name="preprocessed_seg")

/home/weasel/reps/ai-seizure-detectives/source/../data/preprocessed_seg.arrow was successfully written.
